### Atividade Busca Semântica

Nesta atividade você deve aplicar os conhecimentos sobre word embeddings e tokenização para criar um mecanismos de busca semântica. Será disponibilizado um conjunto de dados que possui perguntas médicas, em inglês. Esse conjunto de dados possui as perguntas e também as respostas.

Você deve gerar os vetores das perguntas do conjunto de dados, e permitir que o "usuário" envie a sua pergunta. Você também deve gerar o vetor da pergunta do usuário e com isso buscar a resposta ideal para o usuário. **A resposta ideal é aquela onde o vetor da pergunta do usuário é mais similar ao vetor da pergunta do conjunto de dados** Consulte o notebook "nlp2.ipynb" para verificar como realizamos esse processo

Portanto, no seu script deve ser possível escrever um texto "pergunta" e deve ser retornado a resposta adequada, isto é, a resposta associada a pergunta mais similar no conjunto de dados.

Para isso utilize o pandas e os packages do huggingface

In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from torch import Tensor
from transformers.tokenization_utils_base import BatchEncoding

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## Amostra do conjunto

df = pd.read_csv("data/medquad.csv")
df_amostra = df.sample(5000)

In [3]:
# Modelo para a língua inglesa

nome_modelo = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(nome_modelo)
model = AutoModel.from_pretrained(nome_modelo)

In [4]:
## Funções para: 1) Obter os tokens; 2) Obter os embeddings

def get_tokens(pergunta: str) -> str:
    return tokenizer(pergunta, return_tensors="pt")

def get_vetores(tokens_pergunta: BatchEncoding) -> Tensor:

    with torch.no_grad():
        outputs = model(**tokens_pergunta)
        embeddings = outputs.last_hidden_state

    return embeddings

In [ ]:
## Novas colunas para os tokens e para os embeddings

df_amostra['tokens'] = df_amostra['question'].apply(lambda x: get_tokens(x))
df_amostra['vetores'] = df_amostra['tokens'].apply(lambda x: get_vetores(x))

In [ ]:
# Função para buscar a resposta mais parecida com a pergunta do usuário

def buscar_resposta(pergunta_usuario, df):
    # Gera tokens e vetor para a pergunta do usuário
    tokens_usuario = get_tokens(pergunta_usuario)
    vetor_usuario = get_vetores(tokens_usuario).mean(dim=1).squeeze()  # vetor médio

    # Calcula a similaridade de cosseno com todas as perguntas do dataset
    vetores_dataset = df['vetores'].apply(lambda x: x.mean(dim=1).squeeze())
    similaridades = vetores_dataset.apply(lambda v: F.cosine_similarity(vetor_usuario, v, dim=0))

    # Encontra o índice da pergunta mais parecida
    idx_mais_similar = similaridades.idxmax()
    pergunta_mais_similar = df.loc[idx_mais_similar, 'question']
    resposta = df.loc[idx_mais_similar, 'answer']

    return pergunta_mais_similar, resposta

print("Sistema de busca semântica implementado com sucesso!")
print("Execute a célula seguinte para testar com diferentes perguntas.")

In [ ]:
# Teste com uma pergunta de exemplo
# Mude a pergunta abaixo para testar outras perguntas

pergunta_teste = "What is diabetes?"

pergunta_similar, resposta = buscar_resposta(pergunta_teste, df_amostra)

print(f"Sua pergunta: {pergunta_teste}")
print(f"Pergunta mais similar encontrada: {pergunta_similar}")
print(f"Resposta: {resposta}")